# Topic models

## Dataset for the exercise

* [New York Times Comments](https://www.kaggle.com/aashita/nyt-comments/data), set of reders' comments for articles published in the New York Times

## Overarching research question

The comments allow a perspective to study what kind of concerns people raise when commenting to online articles.
Examine if meaninful themes emerge from the data set.

In [ ]:
## data collection from files.
## to keep the dataset fairly small, we conduct random data selection here.
## this is *ONLY* to ensure that the model is suitable for teaching purposes

import os
import csv
import random

random.seed(1)

path = 'data/nyt-comments/'
files = os.listdir( path ) ## see all files in directory
files = filter( lambda file_name: file_name.startswith("Comments"), files ) ## choose only data files for April
files = map( lambda file_name: path + file_name, files ) ## add path to file names

documents = []

for file in files:
    for entry in csv.DictReader( open( file ) ):
        
        if random.random() > .99: ## choose content randomly
            comment = entry['commentBody']

            documents.append( comment )
            
            
print("Data sample size", len(documents) )

## From text data to document-term matrix

To analyse textual data we transform it to a document term matrix, where in rows we have documents (different comments) and columns represent each word in the dataset.

Note how we **preprocess** the text during this quantification. We remove stopwords (through a set of common English stopwords; we could also create our own lists), stem the content of comments to ensure language is treated well and lower case everything in the content. Thus, the `document_terms` is a huge sparse matrix in the end. Preprocessing is its own kind of art, as it can [influence results](https://www.cambridge.org/core/product/identifier/S1047198717000444/type/journal_article).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.snowball import EnglishStemmer

stemmer = EnglishStemmer()

def stem( text ):
    words = nltk.word_tokenize(text)
    return [ stemmer.stem(w) for w in words ]
    

tf_vectorizer = CountVectorizer(max_df=0.90, min_df=10, stop_words='english', analyzer = stem, lowercase = True)

document_terms = tf_vectorizer.fit_transform(documents)
document_terms_names = tf_vectorizer.get_feature_names()

## From document-term matrix to analysis

Finally we run the Latent Dirichlet Allocation process to our matrix to create topics.
Similar to k-means, we choose the number of topics; there are also other parameters which could be used to _fine tune_ topic models, see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) for details.
When I use these we with their default parameters as none of them solves the challenge that [topic models work on a different abstration level than humans](http://doi.wiley.com/10.1002/asi.23786).

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation( n_components = 5 )
model = lda.fit( document_terms )

In [ ]:
for topic_number, words in enumerate( model.components_ ):
        print( "Topic", topic_number+1 )
        for word in words.argsort()[:-6:-1]:
            print( "\t", document_terms_names[word] )

In [ ]:
## see the distribution of a document to different topics
model.transform( document_terms[0] )

## Tasks

* Compute the distribution of all documents to each topic. Where could you use this?
* Modify the code and examine a few potential topic numbers. What differences can you detect?
* Modify the preprocessing and remove all words which are shorter than four characters. What do you learn now?

## Model evaluation

There are many different approaches to evaluate topic models (see, [1](http://doi.acm.org/10.1145/1553374.1553515), [2](https://journal.fi/politiikka/article/view/79629) for examples).
We can evaluate the suitability of topic models using statistical measurements like loglikelihood, but [some say](http://www.umiacs.umd.edu/~jbg/docs/nips2009-rtl.pdf) that this might be a bad practice - and [others](https://journal.fi/politiikka/article/view/79629) recommend it.
Here we show how to do it.

In [ ]:
model.score( document_terms )

## Tasks

* Evaluate a set of different topics based on this score. Which one would you choose?